In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import random as rand
from math import e
from tqdm import tqdm
from scipy import integrate

In [4]:
#v is a vector cointanining N elements, each of which has two variables: α and s (s = 0 for defection, s = 1 for cooperation)

def game(v,macrosteps,pars):
                 
    N = len(v)
    vec_a_mean=[]           #empty vector to be filled with the mean value of alpha at each for loop
    vec_x=[]                #empty vector to be filled with the fraction of cooperators at each for loop
    check=[]
    s1_vec=v[:,1]
    a1_vec=v[:,0]     
    
    for t in tqdm(macrosteps, desc = 'Progress Bar'):
        
        v = np.transpose(np.array([a1_vec,s1_vec]))
        amean = np.mean(v[:,0])                       #updated mean of alpha
        vec_a_mean = np.append(vec_a_mean, amean)   
        x = np.mean(v[:,1])                           #updated fraction of cooperators
        vec_x = np.append(vec_x, x)        
        cumulative_payoffs=[]  
        pi = np.zeros(N)
        s1_vec=[]
        a1_vec=[]
        
        for player in range(N):
            a1,s1 = v[player][0],v[player][1] 
            s2 = x
            pm1 = pd.DataFrame(np.array([[-pars['p']*pars['cd']*(1-pars['e'])*(1-pars['d'])-(1-a1)*pars['cn'], -pars['p']*pars['cd']*(1-pars['e'])-(1-a1)*pars['cn']],[-pars['p']*pars['cd']*(1-pars['d']), -pars['p']*pars['cd']]]), index = [1, 0], columns=[1, 0])
            pi[player] =  pm1.at[int(s1),int(s2)]
            cumulative_payoffs = np.append(cumulative_payoffs,pi[player])
                
        for pos1 in range(N):
            pos2=rand.randint(N)
            a1,s1 = v[pos1][0],v[pos1][1]         #alpha,strategy of player 1
            s2 = v[pos2][1]                       #strategy of player 2

            pi_1=cumulative_payoffs[pos1]
            pi_2=cumulative_payoffs[pos2]

            delta_pi = pi_2-pi_1              

            prob = 1/(1+e**(-beta*delta_pi))

            a1_new = a1 + pars['gamma']*(2*x-1)*a1*(1-a1)
            
            if a1_new>=1: a1_new=0.999
            if a1_new<=0: a1_new=0.001

            if rand.random()<prob: s1_new = s2
            else: s1_new = s1
            
            s1_vec=np.append(s1_vec,s1_new)
            a1_vec=np.append(a1_vec,a1_new)
        
    return v, vec_x, vec_a_mean, np.mean(s1_vec), np.mean(a1_vec)

In [15]:
#Parameters
pars={} 
pars['cd']=1
pars['e']=0.9
pars['d']=0.5
pars['gamma']=1

beta=1
N=300
n_coop=150
nums = np.zeros(N) 
nums[:n_coop] = 1
xm=[[0]*21 for _ in range(21)]
am=[[0]*21 for _ in range(21)]
macrosteps = list(range(300))
al = np.linspace(0.01,0.99,N)


i=0
for pars['cn'] in np.linspace(0,1,21):    
    j=0
    
    for pars['p'] in np.linspace(0,1,21):
        rand.seed()
        np.random.shuffle(al)
        np.random.shuffle(nums)

        v = np.transpose(np.array([al,nums]))

        r= game(v,macrosteps,pars)
        
        xm[i][j]=r[3]
        am[i][j]=r[4]
        j=j+1
        
    i=i+1

    
pd.DataFrame(xm).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/wellmix/x_wellmix_300_30.tsv", sep=" ",index=False,header=False)
pd.DataFrame(am).to_csv("/home/aantonio/Dropbox (Politecnico Di Torino Studenti)/Internship/wellmix/a_wellmix_300_30.tsv", sep=" ",index=False,header=False)

Progress Bar: 100%|███████████████████████████| 300/300 [00:42<00:00,  7.04it/s]


Progress Bar: 100%|███████████████████████████| 300/300 [00:48<00:00,  6.20it/s]


Progress Bar: 100%|███████████████████████████| 300/300 [00:51<00:00,  5.78it/s]


Progress Bar: 100%|███████████████████████████| 300/300 [00:52<00:00,  5.70it/s]


Progress Bar: 100%|███████████████████████████| 300/300 [00:52<00:00,  5.66it/s]
